# Debugging

Even if you write clear and readable code, even if you fully understand your codes, env if you are very familiar with your model, weird bugs will inevitably appear and you will need to debug them in some way. 

Fortunately, ``BrainPy`` supports debugging with [pdb](https://docs.python.org/3/library/pdb.html)
module or [breakpoint](https://docs.python.org/3/library/functions.html#breakpoint) (The latest version of 
BrainPy removes the support of debugging in IDEs). That is to say, you do not need to resort to using bunch 
of `print` statements to see what's happening in their code. On the contrary, you can work with 
Python’s interactive source code debugger to see the state of any variable in your model.

For the variables you are interested in, you just need to add the ``pdb.set_trace()`` or ``breakpoint()`` after 
the code line. 

In this section, let's take the HH neuron model as an example to illustrate how to debug your
model within BrainPy.

In [1]:
import brainpy as bp
import numpy as np
import pdb

If you want to debug your model, we would like to recommond you to open the ``show_code=True``.

In [2]:
bp.profile.set(show_code=True, jit=False)

Here, the HH neuron model is defined as:

In [3]:
E_Na = 50.
E_K = -77.
E_leak = -54.387
C = 1.0
g_Na = 120.
g_K = 36.
g_leak = 0.03
V_th = 20.
noise = 1.

ST = bp.types.NeuState(
    {'V': -65., 'm': 0.05, 'h': 0.60,
     'n': 0.32, 'spike': 0., 'input': 0.}
)

@bp.integrate
def int_m(m, _t, V):
    alpha = 0.1 * (V + 40) / (1 - np.exp(-(V + 40) / 10))
    beta = 4.0 * np.exp(-(V + 65) / 18)
    return alpha * (1 - m) - beta * m

@bp.integrate
def int_h(h, _t, V):
    alpha = 0.07 * np.exp(-(V + 65) / 20.)
    beta = 1 / (1 + np.exp(-(V + 35) / 10))
    return alpha * (1 - h) - beta * h

@bp.integrate
def int_n(n, _t, V):
    alpha = 0.01 * (V + 55) / (1 - np.exp(-(V + 55) / 10))
    beta = 0.125 * np.exp(-(V + 65) / 80)
    return alpha * (1 - n) - beta * n

@bp.integrate
def int_V(V, _t, m, h, n, I_ext):
    I_Na = (g_Na * np.power(m, 3.0) * h) * (V - E_Na)
    I_K = (g_K * np.power(n, 4.0))* (V - E_K)
    I_leak = g_leak * (V - E_leak)
    dVdt = (- I_Na - I_K - I_leak + I_ext)/C
    return dVdt, noise / C

def update(ST, _t):
    m = np.clip(int_m(ST['m'], _t, ST['V']), 0., 1.)
    h = np.clip(int_h(ST['h'], _t, ST['V']), 0., 1.)
    n = np.clip(int_n(ST['n'], _t, ST['V']), 0., 1.)
    V = int_V(ST['V'], _t, m, h, n, ST['input'])
    
    pdb.set_trace()
    
    spike = np.logical_and(ST['V'] < V_th, V >= V_th)
    ST['spike'] = spike
    ST['V'] = V
    ST['m'] = m
    ST['h'] = h
    ST['n'] = n
    ST['input'] = 0.

HH = bp.NeuType(ST=ST,
                name='HH_neuron',
                steps=update,
                mode='vector')

In this example, we add ``pdb.set_trace()`` after the variables $m$, $h$, $n$ and $V$. 

Then we can create a neuron group, and try to run this neuron model:

In [4]:
group = bp.NeuGroup(HH, geometry=1, monitors=['spike'])
group.run(1000., inputs=('input', 10.))

def NeuGroup0_input_step(ST, input_inp,):
  # "input" step function of NeuGroup0
  ST[5] += input_inp
  


def NeuGroup0_monitor_step(ST, _i, mon_ST_spike,):
  # "monitor" step function of NeuGroup0
  mon_ST_spike[_i] = ST[4]
  


def NeuGroup0_update(ST, _t,):
 # "update" step function of NeuGroup0
 _int_m_m = ST[1]
 _int_m__t = _t
 _int_m_V = ST[0]
 _int_m_alpha = 0.1 * (_int_m_V + 40) / (1 - np.exp(-(_int_m_V + 40) / 10))
 _int_m_beta = 4.0 * np.exp(-(_int_m_V + 65) / 18)
 _dfm_dt = _int_m_alpha * (1 - _int_m_m) - _int_m_beta * _int_m_m
 _int_m_m = 0.1*_dfm_dt + _int_m_m
 _int_m_res = _int_m_m
 m = np.clip(_int_m_res, 0.0, 1.0)
 
 _int_h_h = ST[2]
 _int_h__t = _t
 _int_h_V = ST[0]
 _int_h_alpha = 0.07 * np.exp(-(_int_h_V + 65) / 20.0)
 _int_h_beta = 1 / (1 + np.exp(-(_int_h_V + 35) / 10))
 _dfh_dt = _int_h_alpha * (1 - _int_h_h) - _int_h_beta * _int_h_h
 _int_h_h = 0.1*_dfh_dt + _int_h_h
 _int_h_res = _int_h_h
 h = np.clip(_int_h_res, 0.0, 1.0)
 
 _int_n_n = ST[3]
 _int_n__t = _t
 _

BdbQuit: 